## 개선방안

`-` 상속 구조에 변화

* 부모 유전자를 3개로 증가
* 각 유전자는 문장구조/동사/이외로 나뉘며 해당 구조 하에서 변이가 발생할 수 있음

---

* 동사는 1, 동사가 아니면 0으로 설정
* 부모 간 교차는 가능한 모든 경우의 수로 설정

### Sample 1 test

In [1]:
import pandas as pd
import numpy as np
import os

from algorithm.Genetic_neo import genetic_neo

In [2]:
## load and shuffle
df_sample = pd.read_csv("sample_submission.csv", index_col = 0)
sample = df_sample.text.to_list()[1]
vrbs = ["walk", "give", "jump", "drive", "bake", "sleep", "laugh"] ## verbs in 1st sample

원본 유전자가 존속할 기대값이 2개 정도이도록 조정.

crossover_mutation에서 변이

mutation_chances = 1 -> 1/1/1

> 변이 확률 = 50%로 조정 시 전부 변이하지 않는 개체가 12.5%

사후 변이 : mutation_chances = 2

> 변이 확률 = 50%로 조정 시 전부 변이하지 않는 개체가 25%

최소 32개의 cross_size가 확보되어야 변이 집단의 안정적인 다양성 확보 가능


$12^3 \times 64 = 110,592$ 또는 $16^3 \times 32 = 131,072$로 설정하는 것을 추천함

In [3]:
optimizr = genetic_neo(sample, vrbs, initial_times = 10000, max_stack = 20,
                       cross_size = 32, mutation_chances = 10, crossover_method = "mixture",
                       parents_size = 16, elite_size = 8, batch_size = 512
                       )

optimizr.reputation(rep_times = 100)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [5836.43240734 3004.3379303  4440.12363163 7095.29736056 2980.95798704
 3173.21325947 4011.31741996 3709.86215528 1430.27927301 1481.45686377
 1504.78641475 1558.62998556 1589.37120748 1652.68455865 1718.52002702
 1738.77739729]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

best genome : [np.str_('scrooge drive and walk laugh the night family reindeer mistletoe elf gingerbread bake give advent sleep chimney jump fireplace ornament'), np.float64(1359.4612356710688)]
parents perplexities : [3512.42689284 9928.10596326 8969.29628578 5525.82314053 2267.79324755
 4074.48647773 3512.42689284 4138.65030343 1359.46123567 1430.27927301
 1481.45686377 1504.78641475 1558.62998556 1589.37120748 1652.68455865
 1718.52002702]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

...

parents perplexities : [3223.18409717 7207.03203573 2163.94343385 7095.29736056 4303.51538896
 4106.44307124 6613.54435381 6717.69252302 1359.46123567 1430.27927301
 1481.45686377 1504.78641475 1558.62998556 1589.37120748 1652.68455865
 1718.52002702]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

parents perplexities : [3652.34607365 2980.95798704 1752.41479743 5974.83941328 4106.44307124
 5231.74419736 6360.18344836 5231.74419736 1359.46123567 1430.27927301
 1481.45686377 1504.78641475 1558.62998556 1589.37120748 1652.68455865
 1718.52002702]


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 